# Super Mario Bros AI

Super Mario Bros AI Model built in Python based on [this tutorial](https://youtu.be/2eeYqJ0uBKE)

## Setup Super Mario

First setup the Mario Environment

#### Dependencies

In [1]:
!pip install gym_super_mario_bros nes_py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 22.2 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
  Created wheel for nes-py: filename=nes_py-8.2.1-cp37-cp37m-linux_x86_64.whl size=437020 sha256=521e564292e08690663d2f33413e8d72258a4ed98db5b460e543e040b4aaf5e2
  Stored in directory: /root/.cache/pip/wheels/17/96/0e/22a8c7dbdf412d8e988286f223b223baf0f4ad90c9e699c56d
Successfully built nes-py


In [2]:
from nes_py.wrappers import JoypadSpace  # Import Joypad Wrapper
import gym_super_mario_bros  # Import game
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT # Import simplified controls

It is always important to simplify controls as much as possible when making an AI for a game. See list of controls provided by `SIMPLE_MOVEMENT` below

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

Now, setup `Super Mario` environment

In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')  # Check docs for other versions
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# env.action_space now only has 7 possible actions (the 7 simplified moves)
# env.observation_space shows what's going on in the screen (in our case 240x256 pixels x 3 frames)

## Preprocessing

In this stage, we will *grayscale* the frames. Colored images carry 3 times as much data, which will take too long to process. We will also do *frame stacking*, which kind of gives the AI **memory** (it can see stuff like velocity of enemies, etc...)

In [5]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu


In [6]:
!pip install stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 31.1 MB/s 
     |████████████████████████████████| 1.5 MB 49.5 MB/s 
     |████████████████████████████████| 1.1 MB 53.5 MB/s 
     |████████████████████████████████| 1.6 MB 57.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=ddc78528139a2c31d33fc907994b52b12e0c29b7f1ba09495dfd6f41b8575d3f
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=b7f67667d1ec5ed50976b15ef44b6dca48fcaa88b63fcc8f4de2523ea3d31c7b
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c4

In [7]:
# Import FrameStack Wrapper and Grayscale Wrapper
from gym.wrappers import FrameStack, GrayScaleObservation

# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

# Import matplotlib for visualization
from matplotlib import pyplot as plt



In [8]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

# Train RL Model

In [9]:
# Connect to Google Drive to save old progress

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [11]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [12]:
CHECKPOINT_DIR = './drive/MyDrive/SuperMarioAI/train/'
LOG_DIR = './drive/MyDrive/SuperMarioAI/logs/'

In [13]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [27]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./drive/MyDrive/SuperMarioAI/logs/PPO_3
----------------------------
| time/              |     |
|    fps             | 169 |
|    iterations      | 1   |
|    time_elapsed    | 3   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 99            |
|    iterations           | 2             |
|    time_elapsed         | 10            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 3.7355232e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.00287       |
|    learning_rate        | 1e-06         |
|    loss                 | 211           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000115     |
|    value_loss           | 516           

In [22]:
#@title Utility to zip and download a directory
#@markdown Use this method to zip and download a directory. For ex. a TB logs 
#@markdown directory or a checkpoint(s) directory.

from google.colab import files
import os

dir_to_zip = './drive/MyDrive/SuperMarioAI/train' #@param {type: "string"}
output_filename = './train.zip' #@param {type: "string"}

os.system( "zip -r {} {}".format(output_filename, dir_to_zip ) )

0

In [23]:
model.save("latest"). # Manual Save of latest model

## Test Out Model

In [24]:
# Load our best model
model = PPO.load('latest') # load in model saved at "./latest.zip"

In [ ]:
# Start game
state = env.reset()

# Loop through game
while True:
  action, _state = model.predict(state)
  state, reward, done, info = env.step(action)
  env.render()